In [2]:
using SeisNoise, SeisBase, PyPlot, CUDA, Glob, HDF5, Combinatorics, Random, Statistics, ImageFiltering, FFTW, JLD2, Dates
import SeisNoise: NoiseData 
import SeisBase: read_nodal, NodalData, InstrumentPosition, InstrumentResponse, show_str, show_t, show_x, show_os
import FFTW: rfft, irfft
import DSP: hilbert
import Images: findlocalmaxima
import Base:show, size, summary
import PyCall
import SeisDvv
import Dates
import FiniteDifferences
import CSV
using NetCDF
using DataFrames
include("../correlation/functions/Types.jl")
include("../correlation/functions/Nodal.jl")
include("../correlation/functions/Misc.jl")
include("../correlation/functions/Workflow.jl")
include("../correlation/functions/Dvv.jl")
include("../correlation/functions/Plot.jl")

plot_autocorr_space (generic function with 1 method)

In [ ]:
"

Read and preprocess autocorrelations

"

# frequency bands to return
bands = [[10. 13.];]

# set number of samples and correlations
ns = 801
ncorr = 3090

# processing parameters
cmin,cmax = 3500,4250

# get files and datetimes
files, path, datetimes = get_files_and_datetimes(cmin,cmax)

# restack time
restack_interval = 30

# get autocorrs
restack_times, all_autocorrs = collect_autocorrs(files,datetimes,restack_interval,ns,ncorr,bands)

# save the autocorrelations and datetime information
fname = string(path,"autocorrelations_",Int(bands[1,1]),"-",Int(bands[end,2]),"Hz_",restack_interval,"_min_stack.jld2")
JLD2.save(fname,Dict("autocorrs"=>all_autocorrs))
fname = string(path,"datetimes_",restack_interval,"_min_stack.jld2")
JLD2.save(fname,Dict("datetimes"=>collect(restack_times)))

In [ ]:
"

Visualize the autocorrelation functions in time and space

"

# choose which stacking
restack_interval = 30

# choose frequency bands
bands = [[10. 13.];]

# load p-wave autocorrelations
path = string("/fd1/solinger/correlations/fk_3500_4250/no_whitening/10_min/icequakes_removed/")
fname = string(path,"autocorrelations_",Int(bands[1,1]),"-",Int(bands[end,2]),"Hz_",restack_interval,"_min_stack.jld2")
p_autocorrs = JLD2.load(fname)["autocorrs"]
zero_ind = size(p_autocorrs,1)÷2+1
pos_p_autocorrs = p_autocorrs[zero_ind:end,:,:,:]

# load s-wave autocorrelations
path = string("/fd1/solinger/correlations/fk_1500_2250/no_whitening/10_min/icequakes_removed/")
fname = string(path,"autocorrelations_",Int(bands[1,1]),"-",Int(bands[end,2]),"Hz_",restack_interval,"_min_stack.jld2")
s_autocorrs = JLD2.load(fname)["autocorrs"]
zero_ind = size(s_autocorrs,1)÷2+1
pos_s_autocorrs = s_autocorrs[zero_ind:end,:,:,:]

# choose some time period to skip 
skip_time = [DateTime(2019,7,7,13),DateTime(2019,7,7,14)]
start_time = DateTime(2019,7,6,10)

# make the plot
plot_autocorr_time(pos_p_autocorrs,pos_s_autocorrs,restack_interval,start_time,skip_time,515,path)
plot_autocorr_space(pos_p_autocorrs[:,1:1030,:,:],pos_s_autocorrs[:,1:1030,:,:],restack_interval,start_time,skip_time,path)

In [ ]:
"

Identify data gaps

"

# choose run 
path_1khz = "/1-fnp/petasaur/p-wd03/greenland/Store Glacier DAS data/"
path_resampled = "/1-fnp/pnwstore1/p-wd05/greenland/data/resampled"
files_1khz = glob("1kHz/*",path_1khz)
files_resampled = glob("*",path_resampled)
files = cat(files_1khz,files_resampled,dims=1)

# get datetimes of each file
file_datetimes = []
for i=1:size(files,1)
    file_datetimes = vcat(file_datetimes,DateTime(split(split(files[i],"AQ_")[2],".")[1],dateformat"yymmddHHMMSS"))
end

# get time delta
diff_vec = diff(file_datetimes)
diff_vec_sec = []
for i=1:size(diff_vec,1)
    diff_vec_sec = vcat(diff_vec_sec,diff_vec[i].value/1000)
end

# make a crude plot
fig,ax = plt.subplots(1,1,figsize=(10,5))
plt.scatter(file_datetimes[2:end],diff_vec_sec)
plt.yscale("log")

In [7]:
"

Set common parameters for dv/v calculation

"

# choose frequency bands
bands = [[10. 13.];]
n_bands = size(bands,1)
band = 1

# set gain correction function
gain_correct = "none"

# set parameters for all dv/v methods
method = "stretching"

# set stretching parameters
dvlims = [-0.2,0.2]
ntrial = 100

# choose some time period to skip 
skip_time = [DateTime(2019,7,7,13),DateTime(2019,7,7,14)]

# 1. channelwise autocorrelation functions averaged over the entire time period
# 2. channelwise autocorrelation functions averaged over the last hour
ref_type = ["time_space_avg"] #"time_avg"

# autocorrelation functions averaged over all channels and the entire time period
# surface channel autocorrelation functions averaged over the entire time period

# set parameters for all dv/v runs
clims = [[3500,4250],[1500,2250]]
tlims = [[0.3,0.8],[0.0,0.3]]
restack_intervals = [30] #[60,30,10]
sides = ["pos","neg"]
modes = ["dx"]
parameters = collect(Iterators.product(clims,sides,tlims,restack_intervals,ref_type,modes))

# filter out the unwanted combinations
parameters = filter(e->e[1] == clims[1] && e[3] == tlims[1] || 
                    e[1] == clims[2] && e[3] == tlims[2],parameters)
parameters = filter(e->e[6] == modes[1] && e[5] == ref_type[1] || 
                    e[6] == modes[2] && e[5] == ref_type[2],parameters)

4-element Vector{Tuple{Vector{Int64}, String, Vector{Float64}, Int64, String, String}}:
 ([3500, 4250], "pos", [0.3, 0.8], 30, "time_space_avg", "dx")
 ([3500, 4250], "neg", [0.3, 0.8], 30, "time_space_avg", "dx")
 ([1500, 2250], "pos", [0.0, 0.3], 30, "time_space_avg", "dx")
 ([1500, 2250], "neg", [0.0, 0.3], 30, "time_space_avg", "dx")

In [8]:
"

Compute dv/dt or dv/dx for each set of parameters 
(P wave / S wave, positive / negative lag, coda / ballistic, 60 / 10 minute stack)

"

# iterate through list of parameters
for parameter in parameters
    
    # get parameters
    clim = parameter[1]
    side = parameter[2]
    tlim = parameter[3]
    restack_interval = parameter[4]
    ref_type = parameter[5]
    mode = parameter[6]

    # choose first hour to consider (exclude noisy period at start of deployment)
    start_time = DateTime(2019,7,6,10)

    # load autocorrelations
    path = string("/fd1/solinger/correlations/fk_",clim[1],"_",clim[2],"/no_whitening/10_min/icequakes_removed/")
    fname = string(path,"autocorrelations_",Int(bands[1,1]),"-",Int(bands[end,2]),"Hz_",restack_interval,"_min_stack.jld2")
    autocorrs = JLD2.load(fname)["autocorrs"]

    # get correct side
    zero_ind = size(autocorrs,1)÷2+1
    if side == "pos"
        autocorrs = autocorrs[zero_ind:end,:,:,:]
    elseif side == "neg"
        autocorrs = autocorrs[1:zero_ind,:,:,:]
        autocorrs = reverse(autocorrs,dims=1)
    end

    # read datetimes
    fname = string(path,"datetimes_",restack_interval,"_min_stack.jld2")
    datetimes = JLD2.load(fname)["datetimes"]

    # zero the window to skip
    skip_ind = [findfirst(datetimes .> skip_time[1]),findfirst(datetimes .> skip_time[2])]
    autocorrs[:,:,skip_ind[1]:skip_ind[2]] .= 0

    # get index of first correlation
    start_ind = findfirst(datetimes .> start_time)
    autocorrs = autocorrs[:,:,start_ind:end,:]

    # compute reference waveforms based on selected parameters
    if mode == "dt"
        if ref_type == "time_avg"
            ref_corrs = sum(autocorrs[:,:,:,:],dims=3)
        elseif ref_type == "last"
            last_idx = findall(x -> x == 1, (autocorrs[1,1,:,1] .!= 0))[end]
            ref_corrs = autocorrs[:,:,last_idx:last_idx,:]
        end
    elseif mode == "dx"
        if ref_type == "time_space_avg"
            time_avg_corrs = mean(autocorrs[:,:,(autocorrs[1,1,:,1] .!= 0),1],dims=3)
            time_space_avg_corrs = mean(time_avg_corrs,dims=2)
            ref_corrs = autocorrs[:,:,1:1,:]
            ref_corrs .= time_space_avg_corrs
        end
    end

    # run stretching dvv computation
    if mode == "dt"
        dvv = compute_stretching_dvdt(autocorrs,ref_corrs,bands,tlim,dvlims,ntrial,gain_correct)
    elseif mode == "dx"
        if ref_type == "time_space_avg"
            time_avg_autocorrs = mean(autocorrs[:,:,(autocorrs[1,1,:,1] .!= 0),1],dims=3)
            dvv = compute_stretching_dvdx(time_avg_autocorrs,ref_corrs,bands,tlim,dvlims,ntrial,gain_correct)
        elseif ref_type == "cumulative"
            dvv = compute_stretching_dvdx_cumulative(autocorrs,bands,tlim,dvlims,ntrial,gain_correct)
        end
    end

    # save results
    fname = string(path,"auto_dv",mode,"_",method,"_",Int(bands[1,1]),"-",Int(bands[end,2]),"Hz_",tlim[1],"-",tlim[2],"s_",restack_interval,"_min_",ref_type,"_ref_",side,".jld2")
    JLD2.save(fname,Dict("dvv"=>dvv))

    # give output
    println("Saved ",fname)
end

Saved /fd1/solinger/correlations/fk_3500_4250/no_whitening/10_min/icequakes_removed/auto_dvdx_stretching_10-13Hz_0.3-0.8s_30_min_time_space_avg_ref_pos.jld2
Saved /fd1/solinger/correlations/fk_3500_4250/no_whitening/10_min/icequakes_removed/auto_dvdx_stretching_10-13Hz_0.3-0.8s_30_min_time_space_avg_ref_neg.jld2
Saved /fd1/solinger/correlations/fk_1500_2250/no_whitening/10_min/icequakes_removed/auto_dvdx_stretching_10-13Hz_0.0-0.3s_30_min_time_space_avg_ref_pos.jld2
Saved /fd1/solinger/correlations/fk_1500_2250/no_whitening/10_min/icequakes_removed/auto_dvdx_stretching_10-13Hz_0.0-0.3s_30_min_time_space_avg_ref_neg.jld2


In [ ]:
"

Stack dv/v

"

# choose which fk band
cmin = 3500
cmax = 4250

# choose which time window
t_win = [0.3,0.8]

# choose which stacking
restack_interval = 60

# choose which reference type
ref_type = "time_avg"

# choose which band (probably 1)
band = 1

# choose space or time
mode = "dt"

# set path
path = string("/fd1/solinger/correlations/fk_",cmin,"_",cmax,"/no_whitening/10_min/icequakes_removed/")

# load dv/v from positive autocorrelations
fname = string(path,"auto_dv",mode,"_",method,"_",Int(bands[1,1]),"-",Int(bands[end,2]),"Hz_",t_win[1],"-",t_win[2],"s_",restack_interval,"_min_",ref_type,"_pos.jld2")
auto_dvv_pos = JLD2.load(fname)["dvv"]

# load dv/v from negative autocorrelations
fname = string(path,"auto_dv",mode,"_",method,"_",Int(bands[1,1]),"-",Int(bands[end,2]),"Hz_",t_win[1],"-",t_win[2],"s_",restack_interval,"_min_",ref_type,"_neg.jld2")
auto_dvv_neg = JLD2.load(fname)["dvv"]

# stack downgoing, upgoing, and cross autocorrelations
auto_dvv = auto_dvv_pos[1:1030,:,band] .+ auto_dvv_pos[1031:2060,:,band] .+ auto_dvv_pos[2061:3090,:,band]
auto_dvv = auto_dvv .+ auto_dvv_neg[1:1030,:,band] .+ auto_dvv_neg[1031:2060,:,band] .+ auto_dvv_neg[2061:3090,:,band]
auto_dvv = auto_dvv ./ 6

# read stack datetime info
fname = string(path,"datetimes_",restack_interval,"_min_stack.jld2")
datetimes = JLD2.load(fname)["datetimes"]

# if mode = dx, cumulatively add 
if mode == "dx"
    auto_dvv_sum = similar(auto_dvv)
    for i=1:size(auto_dvv,1)
        auto_dvv_sum[i,:] = sum(auto_dvv[1:i,:],dims=1)
    end
    auto_dvv = auto_dvv_sum
end

In [ ]:
"

Visualize dv/v

"

# get time axis in local time
datestring = []
for i=1:size(datetimes,1)
    datestring = vcat(datestring,Dates.format(datetimes[i],"mm-dd\nHH:MM"))
end

# make color plot 
fig,ax = plt.subplots(1,1,figsize=(10,4))
c1 = ax.imshow(auto_dvv[:,:,band],aspect="auto",cmap="seismic",vmin=-30,vmax=30)
ax.set_ylabel("Depth")
ax.set_xlabel("Time")

# handle date axis
start_time = DateTime(2019,7,6,10)
start_ind = findfirst(datetimes .> start_time)
datelabels = [start_time+Hour(6*i) for i=0:9]
datestring = []
for i=1:size(datelabels,1)
    datestring = vcat(datestring,Dates.format(datelabels[i],"mm-dd\nHH:MM"))
end

# set xlims on all plots
xticks = (datelabels.-datetimes[start_ind])./(datetimes[end].-datetimes[start_ind]).*size(auto_dvv,2)
ax.set_xticks(xticks)
ax.set_xticklabels(datestring)
lim_ind = [findfirst(abs.(auto_dvv[1,:,1]) .> 0),size(auto_dvv,2)-1]
ax.set_xlim(lim_ind)

title = string("Autocorrrelation dv/",mode," \n(",method,", ",cmin,"-",cmax,"m/s, ",Int(bands[band,1]),"-",Int(bands[band,2]),"Hz, ",t_win[1],"-",t_win[2],"s, ",restack_interval," min stack)")
ax.set_title(title)
plt.colorbar(c1)
plt.tight_layout()

fname = string("dv",mode,"_",method,"_fk_",cmin,"_",cmax,"_",Int(bands[band,1]),"-",Int(bands[band,2]),"Hz_",t_win[1],"-",t_win[2],"s_",restack_interval,"_min_",ref_type,"_ref_stack.png")
plt.savefig("../figures/dvv/"*fname,dpi=100)
plt.show()

In [ ]:
"

Make superplot with all the data

"

# choose which phase
phase = "s"

# choose which stacking
restack_interval = 10

# select other parameters accordingly
if phase == "p"
    # choose which fk band
    cmin = 3500
    cmax = 4250

    # choose which time window
    t_win = [0.3,0.8]
    
elseif phase == "s"
    # choose which fk band
    cmin = 1500
    cmax = 2250

    # choose which time window
    t_win = [0.0,0.3]
end

# choose which reference type
ref_type = "time_avg"

# choose start time
start_time = DateTime(2019,7,6,10)

# choose which band (probably 1)
band = 1

# choose space or time
mode = "dt"

# set path
path = string("/fd1/solinger/correlations/fk_",cmin,"_",cmax,"/no_whitening/10_min/icequakes_removed/")

# load dv/v from positive autocorrelations
fname = string(path,"auto_dv",mode,"_stretching_",Int(bands[1,1]),"-",Int(bands[end,2]),"Hz_",t_win[1],"-",t_win[2],"s_",restack_interval,"_min_",ref_type,"_pos.jld2")
auto_dvv_pos = JLD2.load(fname)["dvv"]

# load dv/v from negative autocorrelations
fname = string(path,"auto_dv",mode,"_stretching_",Int(bands[1,1]),"-",Int(bands[end,2]),"Hz_",t_win[1],"-",t_win[2],"s_",restack_interval,"_min_",ref_type,"_neg.jld2")
auto_dvv_neg = JLD2.load(fname)["dvv"]

# stack downgoing, upgoing, and cross autocorrelations
auto_dvv = auto_dvv_pos[1:1030,:,band] .+ auto_dvv_pos[1031:2060,:,band] .+ auto_dvv_pos[2061:3090,:,band]
auto_dvv = auto_dvv .+ auto_dvv_neg[1:1030,:,band] .+ auto_dvv_neg[1031:2060,:,band] .+ auto_dvv_neg[2061:3090,:,band]
auto_dvv = auto_dvv ./ 6

# read stack datetime info
fname = string(path,"datetimes_",restack_interval,"_min_stack.jld2")
datetimes = JLD2.load(fname)["datetimes"]

# read borehole pressure
if phase == "p"
    p_file = "../borehole_data/BH19e_pressure.csv"
elseif phase == "s"
    p_file = "../borehole_data/BH19c_pressure.csv"
end
p_BH19 = DataFrame(CSV.File(p_file))
format = "yyyy-mm-dd HH:MM:SS"
t_BH19 =  [DateTime(p_BH19[i,1],format) for i in range(1,size(p_BH19,1))]

# read ice velocity
gnss = DataFrame(CSV.File("../borehole_data/R30_GNSS.csv"))
t_gnss = [DateTime(gnss[i,1],format) for i in range(1,size(gnss,1))]

# set time offset for borehole data from later
shift = Dates.Day(23)

# detrend pressure within time period of interest
if phase == "p"
    t_p_shift = t_BH19 - shift
    p_shift = p_BH19[:,3][(t_p_shift .< datetimes[end]) .&& (t_p_shift .> start_time)]
    t_p_shift = t_p_shift[(t_p_shift .< datetimes[end]) .&& (t_p_shift .> start_time)]
    detrend!(p_shift)
elseif phase == "s"
    p_shift = p_BH19[:,3][(t_BH19 .< datetimes[end]) .&& (t_BH19 .> start_time)]
    t_p_shift = t_BH19[(t_BH19 .< datetimes[end]) .&& (t_BH19 .> start_time)]
    detrend!(p_shift)
end

# detrend pressure within time period of interest 
t_v_shift = t_gnss - shift
v_shift = gnss[:,6][(t_v_shift .< datetimes[end]) .&& (t_v_shift .> start_time)]
t_v_shift = t_v_shift[(t_v_shift .< datetimes[end]) .&& (t_v_shift .> start_time)]
detrend!(v_shift)

# plot borehole pressure
fig,ax = plt.subplots(3,1,figsize = (13,9),gridspec_kw=Dict("height_ratios" => [1,3,1]))

# handle some date stuff
mdates = PyCall.pyimport(("matplotlib.dates"))
date_format = mdates.DateFormatter("%m-%d\n%H:%M")

# plot surface velocity
ax[1].set_title("A.",loc="left",size=15)
ax[1].plot(t_v_shift+shift,v_shift/365,label="Velocity",c="k",linewidth=4)
ax[1].set_ylabel("Velocity perturbation \n(m/day)",size=15)
ax[1].set_xlim(start_time+shift,datetimes[end]+shift)
ax[1].grid(true) 
ax[1].xaxis.set_major_formatter(date_format)

# plot borehole pressure
if phase == "p"
    ax[3].plot(t_p_shift+shift,p_shift,c="k",linewidth=6)
    ax[3].plot(t_p_shift+shift,p_shift,c="#95EC3A",linewidth=4)
    ax[3].set_xlim(start_time+shift,datetimes[end]+shift)
elseif phase == "s"
    ax[3].plot(t_p_shift,p_shift,c="k",linewidth=6)
    ax[3].plot(t_p_shift,p_shift,c="#E86E10",linewidth=4)
    ax[3].set_xlim(start_time,datetimes[end])
end
ax[3].set_ylabel("Hydraulic head \nperturbation (m)",size=15)
ax[3].set_title("C.",loc="left",size=15)
ax[3].grid(true)  
ax[3].xaxis.set_major_formatter(date_format)

# plot stacked dv/v
if phase == "p"
    lim = 0.75
elseif phase == "s"
    lim = 1.5
end
extent = [0, size(auto_dvv,2), size(auto_dvv,1)*1.017, 0]
c1 = ax[2].imshow(auto_dvv[:,:,band],origin="upper",aspect="auto",cmap="seismic",extent=extent,vmin=-lim,vmax=lim,interpolation="none")
ax[2].set_ylabel("Depth",size=15)
ax[2].set_title("B.",loc="left",size=15)

# handle date axis
datelabels = [DateTime(2019,7,6,12)+Hour(6*i) for i=0:9]
datestring = []
for i=1:size(datelabels,1)
    datestring = vcat(datestring,Dates.format(datelabels[i],"mm-dd\nHH:MM"))
end

# get timing info for x axis
xticks = (datelabels.-start_time)./(datetimes[end].-start_time).*size(auto_dvv,2)
ax[2].set_xticks(xticks)
ax[2].set_xticklabels(datestring)
ax[2].grid(true)  

# colorbar
box = ax[2].get_position()
axColor = plt.axes([box.x0*1.5 + box.width*1.05, box.y0-0.025, 0.02, box.height*1.18])
cbar = plt.colorbar(c1, cax = axColor, orientation="vertical")
cbar.set_label("dv/v (%)",size=15)

plt.tight_layout()
fname = string("dvv_stretching_fk_",cmin,"_",cmax,".png")
plt.savefig("../paper_figures/"*fname,dpi=100)
plt.show()

In [ ]:
"

Design a time-dependent gain correction

"

# read autocorrealtions
fname = string(path,"autocorrelations_",Int(bands[1,1]),"-",Int(bands[end,2]),"Hz.jld2")
all_autocorrs = JLD2.load(fname)["autocorrs"]
zero_ind = size(all_autocorrs,1)÷2+1
pos_autocorrs = all_autocorrs[zero_ind:end,:,:,:]

# manually fit an exponential function describing the envelope decay
lambda = 0.04
exp_func = exp.(-lambda*range(1,size(pos_autocorrs,1)))

# make a gain correction function
pre_trim = 40
post_trim = pre_trim-1
exp_func_pre = reverse(exp_func)[pre_trim:end] .- reverse(exp_func)[end]
gain_correction = 1 .+ cat(exp_func_pre,(1 .- exp_func[1:post_trim]),dims=1)
gain_correction = gain_correction .- gain_correction[1]

# choose channel (eventually, we'll do all)
chan_idx = 500

# get average autocorrelation
start_ind = 14
ref_s = sum(pos_autocorrs[:,chan_idx,start_ind:end,1],dims=2)
ref_s = ref_s./(size(pos_autocorrs,3)-start_ind+1)
t = range(0,1,size(pos_autocorrs,1))

# plot result
fig,ax = plt.subplots(3,1,figsize=(7,10))
ax[1].plot(t,ref_s)
ax[1].set_title("Reference waveform (channel 500)")
ax[1].set_ylabel("Amplitude")

# plot the gain-corrected result
ax[2].plot(t,gain_correction)
ax[2].set_title("Gain correction function")
ax[2].set_ylabel("Gain factor")
ax[3].plot(t,ref_s)
ax[3].set_ylabel("Amplitude")
ax[3].set_title("Gain-corrected reference waveform (channel 500)")
ax[3].set_xlabel("Time (s)")
plt.title("Gain-corrected reference waveform")
plt.tight_layout()
plt.show()

In [ ]:
"

Calculate dv/v for one channel of autocorrelations

"

# choose channel and frequency band 
c = 70
j = 1

# get reference waveform (gain-corrected average autocorrelation at this channel)
ref_s = mean(pos_autocorrs[:,c,start_ind:end],dims=2)
ref_s = gain_correction .* ref_s
ref_s = ref_s .- mean(ref_s)
ref_s = ref_s ./ maximum(abs.(ref_s))
ref_s = dropdims(ref_s,dims=2)

# iterate through each hour
dvv = zeros((start_ind:size(pos_autocorrs,3)))
for i=start_ind:size(pos_autocorrs,3)

    # get gain-corrected waveform to compare with reference
    s = pos_autocorrs[:,c,i] .* gain_correction
    s = s .- mean(s)
    s = s ./ maximum(abs.(s))
    
    # calculate dv/v with MWCS
    dvv[i],cc,cdp,ϵ,err,allC = SeisDvv.stretching(ref_s,s,t,t_window,10.,25.,dvmin=dvmin,dvmax=dvmax,ntrial=ntrial)
end

# plot the result
plt.plot(dvv)
print(mean(dvv))

In [ ]:
"

Extract adjacent / surrounding cross correlations for dv/v stacking

"

# list all correlation files
path = string("/fd1/solinger/correlations/fk_",cmin,"_",cmax,"/no_whitening/")
files = glob("*0.jld2",path)
files = files[BitVector(1 .- contains.(files,"error"))]
files = files[BitVector(1 .- contains.(files,"autocorrelation"))]

# set number of channels on each side to include
chan_bin_width = 5
num_hours = size(files,1)

# make output container
stack_corrs = zeros(803,chans[2]-chans[1]+1,chan_bin_width,num_hours)

# get indicies for correlations
chan_pairs = collect(with_replacement_combinations(collect(chans[1]:chans[2]),2))
chan_pairs = reduce(vcat,transpose.(chan_pairs))

# get indices of correlation functions
for i = 1:size(files,1)

    # read and filter
    C = JLD2.load(files[i])["NodalCorrData"]
    C_filt = deepcopy(C)
    clean_up!(C_filt,freqmin,freqmax)
    abs_max!(C_filt)

    for c = 1:chans[2]-chans[1]+1
        chan = chans[1]+c-1
        for j in 1:chan_bin_width
            corr_ind = ((chan_pairs[:,1] .== chan-j) .&& (chan_pairs[:,2] .== chan+j))
            if sum(corr_ind) == 1
                corr = C_filt.corr[:,corr_ind]
                stack_corrs[:,c,j,i] = corr
            end
        end
    end
end

# save the correlations
fname = string(path,"adjacent_correlations_",freqmin,"-",freqmax,"Hz.jld2")
JLD2.save(fname,Dict("corrs"=>stack_corrs))

In [ ]:
"

Calculate dv/v for adjacent correlations

"



# choose if computing dv/v in reference to autocorrelation stack or adjacent correlation stack
ref = "adj"

# make output containers
dvv = zeros(size(pos_corrs,2),size(pos_corrs,3),size(pos_corrs,4),n_bands)
dvv0 = zeros(size(pos_corrs,2),size(pos_corrs,3),size(pos_corrs,4),n_bands)

# iterate through channel
for c=1:size(pos_corrs,2)
    
    # get reference waveform (gain-corrected average autocorrelation at this channel)
    ref_s = 0
    if ref == "auto"
        ref_s = sum(pos_autocorrs[:,c,start_ind:end],dims=2)
        ref_s = ref_s./(size(pos_autocorrs,3)-start_ind+1)
        ref_s = gain_correction .* ref_s
        ref_s = ref_s .- mean(ref_s)
        ref_s = ref_s ./ maximum(abs.(ref_s))
    end
    
    # iterate through each hour
    for i=start_ind:size(pos_corrs,4)
        
        # iterate through each adjacent correlation
        for k = 1:size(pos_corrs,3)
            
            # get reference waveform (gain-corrected average adjacent correlation for this channel pair)
            if ref == "adj"
                ref_s = sum(pos_corrs[:,c,k,start_ind:end],dims=2)
                ref_s = ref_s./(size(pos_autocorrs,4)-start_ind+1)
                ref_s = gain_correction .* ref_s
                ref_s = ref_s .- mean(ref_s)
                ref_s = ref_s ./ maximum(abs.(ref_s))
            end
            
            # get gain-corrected waveform to compare with reference
            s = pos_corrs[:,c,k,i] .* gain_correction
            s = s .- mean(s)
            s = s ./ maximum(abs.(s))

            # check if empty (will be first and last few)
            if sum(s) != 0 && ~isnan(sum(s))

                # iterate through each frequency band
                for j=1:n_bands

                    # calculate dv/v with MCWS
                    if method == "mwcs"
                        t_mwcs,dt_mwcs,err_mwcs,mcoh_mwcs = SeisDvv.mwcs(ref_s,s,bands[j,1],bands[j,2],fs,t[t_window[1]], win_len, win_step, 0);
                        dvv[c,k,i,j],dvv_err_mwcs,int_mwcs,int_err_mwcs,dvv0[c,k,i+start_ind-1,j],dvv0_err_mwcs = SeisDvv.mwcs_dvv(t_mwcs,dt_mwcs,err_mwcs,mcoh_mwcs,"static",0.0,0.0,t[t_window[1]],t[t_window[end]]-t[t_window[1]],"right");
                    elseif method == "stretching"
                        dvv[c,k,i,j],cc_ts,cdp_Ts,eps_ts,err_ts,allC_ts = SeisDvv.stretching(ref_s,s,t,t_window,bands[j,1],bands[j,2],dvmin=dvmin,dvmax=dvmax,ntrial=ntrial);
                    end
                end
            end
        end
    end
end

# save results
fname = string(path,"adjacent_dvv_",method,"_ref_",ref,"_",freqmin,"-",freqmax,"Hz.jld2")
if method == "mwcs"
    JLD2.save(fname,Dict("dvv"=>dvv,"dvv0"=>dvv0))
elseif method == "stretching"
    JLD2.save(fname,Dict("dvv"=>dvv))
end

In [ ]:
"

Visualize dv/v with enclosing correlations

"

# load dv/v from autocorrelation
path = string("/fd1/solinger/correlations/fk_",cmin,"_",cmax,"/no_whitening/")
fname = string(path,"auto_dvv_",method,"_",freqmin,"-",freqmax,"Hz.jld2")
auto_dvv = JLD2.load(fname)["dvv"]

# load dv/v from adjacent cross-correlations
path = string("/fd1/solinger/correlations/fk_",cmin,"_",cmax,"/no_whitening/")
fname = string(path,"adjacent_dvv_",method,"_ref_adj_",freqmin,"-",freqmax,"Hz.jld2")
corr_dvv = JLD2.load(fname)["dvv"]

# choose what we're plotting
chan = 300
band = 1
n_corr = 5
weight = "proportional"

# stack dv/v for adjacent correlations and autocorrelation
stacked_corr_dvv = dropdims(sum(corr_dvv[:,1:n_corr,:,:],dims=2),dims=2)
if weight == "equal"
    stacked_dvv = (stacked_corr_dvv+auto_dvv)./(n_adj+1)
elseif weight=="proportional"
    stacked_dvv = (stacked_corr_dvv./n_corr+auto_dvv)./2
end

# plot dv/v from autocorrelation and adjancent cross correlations (ref auto) for a particular channel
fig,ax = plt.subplots(figsize=(10,5))
title = string("dv/v stacking for channel at ",chan,"m depth from ",bands[band,1],"-",bands[band,2]," Hz")
plt.suptitle(title,fontsize=15)
ax.plot(auto_dvv[chan,:,band],c="r",alpha=0.5,label="Autocorr dv/v")
ax.plot(corr_dvv[chan,1,:,band],c="k",alpha=0.1,label="Xcorr dv/v")
ax.plot(corr_dvv[chan,2,:,band],c="k",alpha=0.1)
ax.plot(corr_dvv[chan,3,:,band],c="k",alpha=0.1)
ax.plot(corr_dvv[chan,4,:,band],c="k",alpha=0.1)
ax.plot(corr_dvv[chan,5,:,band],c="k",alpha=0.1)
ax.plot(stacked_dvv[chan,:,band],c="k",label="Stacked dv/v")
ax.legend(loc="upper left")
ax.set_ylabel("dv/v")
ax.set_title("Stacked dv/v of autocorrelation and enclosing cross correlations")
plt.show()

# make color plot 
fig,ax = plt.subplots(3,1,figsize=(10,12))
c1 = ax[1].imshow(auto_dvv[:,:,band],aspect="auto",cmap="seismic",vmin=-10,vmax=10)
ax[1].set_ylabel("Depth")
ax[1].set_title("Autocorrrelation dv/v")
plt.colorbar(c1,ax=ax[1])

c2 = ax[2].imshow(stacked_corr_dvv[:,:,band]/n_corr,aspect="auto",
             cmap="seismic",vmin=-20,vmax=20)
ax[2].set_ylabel("Depth")
ax[2].set_title("Stacked dv/v of "*string(n_corr)*" enclosing cross correlations")
plt.colorbar(c2,ax=ax[2])

c3 = ax[3].imshow(stacked_dvv[:,:,band],aspect="auto",cmap="seismic",vmin=-20,vmax=20)
ax[3].set_ylabel("Depth")
ax[3].set_title("Stacked dv/v of autocorrelation and "*string(n_corr)*" enclosing cross correlations")
plt.colorbar(c3,ax=ax[3])
title = string("Various dv/v configurations (",bands[band,1],"-",bands[band,2]," Hz)\n")
plt.suptitle(title,fontsize=15)
plt.tight_layout()
fname = string("dvv_",method,"_",bands[band,1],"-",bands[band,2],"Hz_",weight,"_weight.png")
#plt.savefig("../figures/dvv/"*fname,dpi=100)
plt.show()

In [ ]:
"

OLD Calculate dv/v for autocorrelations

"
t_window = findall(t->t<=t_lims[2] && t>=t_lims[1], t);

# make output containers
dvv = zeros(size(autocorrs,2),size(autocorrs,3),n_bands)
dvv0 = zeros(size(autocorrs,2),size(autocorrs,3),n_bands)

# get number of empty stacks
num_empty = sum(autocorrs[1,1,:,1] .== 0)

# iterate through channel
for c=1:size(autocorrs,2)
    
    # iterate through each frequency band
    for j=1:n_bands
            
        # get reference waveform (gain-corrected average autocorrelation at this channel)
        ref_s = sum(autocorrs[:,c,start_ind:end,j],dims=2)/(size(autocorrs,3)-start_ind+1-num_empty)
        if gain_correct == true
            ref_s = gain_correction .* ref_s
        end
        ref_s = ref_s .- mean(ref_s)
        ref_s = ref_s ./ maximum(abs.(ref_s))
        
        # iterate through each hour
        for i=start_ind:size(autocorrs,3)
            
            # check if empty
            if sum(autocorrs[:,c,i,j]) != 0

                # get gain-corrected waveform to compare with reference
                s = autocorrs[:,c,i,j]
                if gain_correct == true
                    s = s.* gain_correction
                end
                s = s .- mean(s)
                s = s ./ maximum(abs.(s))
                print(s[t_window])
                # calculate dv/v with MWCS
                if method == "mwcs"
                    t_mwcs,dt_mwcs,err_mwcs,mcoh_mwcs = SeisDvv.mwcs(ref_s,s,bands[j,1],bands[j,2],fs,t[t_window[1]], win_len, win_step, 0);
                    dvv[c,i,j],dvv_err_mwcs,int_mwcs,int_err_mwcs,dvv0[c,i+start_ind-1,j],dvv0_err_mwcs = SeisDvv.mwcs_dvv(t_mwcs,dt_mwcs,err_mwcs,mcoh_mwcs,"static",0.0,0.0,t[t_window[1]],t[t_window[end]]-t[t_window[1]],"right");
                elseif method == "stretching"
                    dvv[c,i,j],cc_ts,cdp_Ts,eps_ts,err_ts,allC_ts = SeisDvv.stretching(ref_s,s,t,t_window,bands[j,1],bands[j,2],dvmin=dvmin,dvmax=dvmax,ntrial=ntrial);
                end
            end
        end
    end
end

# save results
fname = string(path,"auto_dvv_",method,"_",Int(bands[1,1]),"-",Int(bands[end,2]),"Hz_",t_win[1],"-",t_win[2],"s_",restack,"_min_",side,".jld2")
if method == "mwcs"
    JLD2.save(fname,Dict("dvv"=>dvv,"dvv0"=>dvv0))
elseif method == "stretching"
    JLD2.save(fname,Dict("dvv"=>dvv))
end


